In [1]:
from PIL import Image
from glob import glob
from tqdm import tqdm
import numpy as np
import os
import pickle
import cv2
import matplotlib.pyplot as plt
import helper_functions as hlf


%matplotlib inline

# Process Damselflies Images

In [11]:
# create new, process, and save images damselfly images
inpath = r'E:\images2019\train_val2019\Damselflies\\'
outpath = r'E:\classification_damsel_images'
dim_tuple = (256, 256)

hlf.process_images(inpath, outpath, dim_tuple)

100%|██████████████████████████████████████████████████████████████████████████████| 8462/8462 [09:00<00:00, 15.67it/s]


# Process Dragonflies Images

In [13]:
# create new, process, and save images dragonfly images
inpath = r'E:\images2019\train_val2019\Dragonflies\\'
outpath = r'E:\classification_dragon_images'
dim_tuple = (256, 256)

hlf.process_images(inpath, outpath, dim_tuple, extension)

100%|██████████████████████████████████████████████████████████████████████████████| 9204/9204 [08:32<00:00, 17.96it/s]


# Train/Test Dragonfly Images

In [ ]:
# create dragonfly train/test sets
inpath = r'E:\classification_dragon_images\\'
dragon_test_path = r'E:\classification_dragon_images\test\dragon_test.npy'
dragon_train_path = r'E:\classification_dragon_images\train\dragon_train.npy'

# define test set size (25%~)
files = os.listdir(inpath) 
n_files = len(files)
n_train = int(n_files * 0.75)

In [14]:
# create dragonfly training set
hlf.save_train_test(inpath, dragon_train_path, extension, 0, n_train)

100%|██████████████████████████████████████████████████████████████████████████| 13807/13807 [00:13<00:00, 1040.84it/s]


In [15]:
# create dragonfly testing set
hlf.ave_train_test(inpath, dragon_test_path, extension, n_train, -1)

100%|█████████████████████████████████████████████████████████████████████████████| 4600/4600 [00:09<00:00, 484.16it/s]


# Train/Test Damselfly Images

In [ ]:
# create damselfly train/test sets
inpath = r'E:\classification_damsel_images\\'
damsel_test_path = r'E:\classification_damsel_images\test\damsel_test.npy'
damsel_train_path = r'E:\classification_damsel_images\train\damsel_train.npy'

# define test set size (25%~)
files = os.listdir(inpath)
n_files = len(files)
n_train = int(n_files * 0.75)

In [16]:
# create dragonfly training set
hlf.save_train_test(inpath, damsel_train_path, extension, 0, n_train)

100%|███████████████████████████████████████████████████████████████████████████| 12694/12694 [00:40<00:00, 316.83it/s]


In [17]:
# create damselfly testing set
hlf.save_train_test(inpath, damsel_test_path, extension, n_train, -1)

100%|█████████████████████████████████████████████████████████████████████████████| 4229/4229 [00:15<00:00, 281.17it/s]
